In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [34]:
productionGE_2020 = pd.read_excel('data\production_by_sector\eCO2mix_RTE_Grand-Est_Annuel-Definitif_2020.xlsx')
productionGE_2020.head()

,Périmètre,Nature,Date,Heures,Consommation,Thermique,Nucléaire,Eolien,Solaire,Hydraulique,...,TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%)
0,Grand-Est,Données définitives,2020-01-01,00:00:00,5703.0,1503.0,11978.0,1145.0,0.0,1104.0,...,210.03,95.21,20.08,31.69,0.0,0.0,19.36,20.59,2.14,41.22
1,Grand-Est,Données définitives,2020-01-01,00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Grand-Est,Données définitives,2020-01-01,00:30:00,5620.0,1509.0,12092.0,1045.0,0.0,953.0,...,215.16,96.12,18.59,28.92,0.0,0.0,16.96,17.77,2.14,40.54
3,Grand-Est,Données définitives,2020-01-01,00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Grand-Est,Données définitives,2020-01-01,01:00:00,5471.0,1380.0,11947.0,846.0,0.0,942.0,...,218.37,94.97,15.46,23.42,0.0,0.0,17.22,17.57,2.23,41.22


In [ ]:
# display missing value on a plot

missing_values=